# Chopper Audio Generation Collaborative Notebook

This notebook allows for collaborative training and inference with the Chopper audio generation project.

In [ ]:
# Clone the repository (your friend will do this)
!git clone https://github.com/Emmanuelekpeh/Chopper.git
%cd Chopper

# Install dependencies
!pip install -r requirements.txt
!pip install torch librosa soundfile matplotlib

In [ ]:
# Import necessary modules
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio

# Import project modules
from core.transformer_generator import TransformerSampleGenerator
from core.improved_generator import ImprovedSampleGenerator
from core.processing_pipeline import ProcessingPipeline

## Training Section

Upload audio samples or download from sources, then train the model.

In [ ]:
# Option to upload samples
from google.colab import files

def upload_samples():
    uploaded = files.upload()
    sample_paths = []
    for filename in uploaded.keys():
        path = f'data/raw/{filename}'
        os.makedirs('data/raw', exist_ok=True)
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        sample_paths.append(path)
    return sample_paths

# Uncomment to upload your samples
# sample_paths = upload_samples()

In [ ]:
# Or download samples from Looperman or Splice
from core.playwright_scraper import PlaywrightScraper
from core.splice_scraper import SpliceLoopScraper

def download_looperman(query="drum loop", pages=2):
    os.makedirs('data/raw', exist_ok=True)
    scraper = PlaywrightScraper(download_dir="data/raw")
    samples = scraper.bulk_download(query, pages=pages)
    print(f"Downloaded {len(samples)} samples from Looperman")
    return samples

# Uncomment to download samples
# sample_paths = download_looperman(query="drum loop", pages=2)

In [ ]:
# Process the samples
def process_samples(sample_paths):
    pipeline = ProcessingPipeline()
    results = pipeline.process_batch(sample_paths)
    metadata = pipeline.create_dataset_metadata(results)
    print(f"Processed {metadata['successful']} files successfully")
    print(f"Created {metadata['total_segments']} segments")
    return metadata

# Uncomment to process your samples
# metadata = process_samples(sample_paths)

In [ ]:
# Train the Transformer model
def train_transformer_model(dataset_paths, epochs=50, batch_size=16, save_path='models/transformer_gan.pt'):
    os.makedirs('models', exist_ok=True)
    generator = TransformerSampleGenerator()
    generator.train(dataset_paths, epochs=epochs, batch_size=batch_size, save_path=save_path)
    return generator

# Uncomment to train with your processed samples
# generator = train_transformer_model(metadata['segment_paths'], epochs=50)

## Inference Section

Generate new audio samples using the trained model.

In [ ]:
# Load a saved model and generate samples
def generate_samples(model_path='models/transformer_gan.pt', count=3):
    generator = TransformerSampleGenerator(model_path=model_path)
    
    os.makedirs('output', exist_ok=True)
    for i in range(count):
        output_path = f'output/generated_sample_{i}.wav'
        generator.generate_and_save_audio(output_path)
        
        # Display and play the audio
        waveform, sr = librosa.load(output_path, sr=None)
        plt.figure(figsize=(10, 3))
        plt.plot(waveform)
        plt.title(f"Generated Sample {i}")
        plt.show()
        
        display(Audio(output_path))
    
    return [f'output/generated_sample_{i}.wav' for i in range(count)]

# Uncomment to generate samples
# sample_files = generate_samples()

## Model Saving & Sharing

Save your model to Google Drive or push it back to GitHub.

In [ ]:
# Save to Google Drive
from google.colab import drive

def save_to_drive(model_path='models/transformer_gan.pt'):
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/chopper_models/'
    os.makedirs(drive_path, exist_ok=True)
    
    import shutil
    dest_path = os.path.join(drive_path, os.path.basename(model_path))
    shutil.copy(model_path, dest_path)
    print(f"Model saved to Google Drive at: {dest_path}")

# Uncomment to save to drive
# save_to_drive()

In [ ]:
# Push changes back to GitHub (if you have permission)
def push_to_github(commit_message="Updated model"):
    # Configure Git
    !git config --global user.email "your-email@example.com"  # Replace with your email
    !git config --global user.name "Your Name"  # Replace with your name
    
    # Add changes
    !git add models/*.pt
    
    # Commit and push
    !git commit -m "{commit_message}"
    !git push origin master
    
    print("Changes pushed to GitHub")

# Uncomment to push changes
# push_to_github("Updated transformer model with new drums")